In [1]:
# Change directory to the root of the project
import os 
os.chdir('..')
print(f"Working directory: {os.getcwd()}")

Working directory: /Users/eohjelle/Documents/2025-dots-and-boxes/dots-and-boxes


In this notebook we will run another wandb sweep for a transformer based model using training data generated with a Minimax model.

In [2]:
# Load the training data

from core.data_structures import ReplayBuffer
import random

buffer = ReplayBuffer.from_file('applications/tic_tac_toe/training_data/transformer.pkl')

print(f"Buffer size: {buffer.states.shape[0]}")
for i in random.sample(range(buffer.states.shape[0]), 1):
    print(f"Buffer state {i}: {buffer.states[i]}")
    for key in buffer.targets.keys():
        print(f"Buffer target {key} {i}: {buffer.targets[key][i]}")
    for key in buffer.data.keys():
        print(f"Buffer data {key} {i}: {buffer.data[key][i]}")

Buffer size: 5478
Buffer state 1912: tensor([2, 1, 1, 2, 0, 1, 0, 0, 2], device='mps:0')
Buffer target policy 1912: tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.3333, 0.0000, 0.3333, 0.3333, 0.0000],
       device='mps:0')
Buffer target value 1912: tensor([-1.], device='mps:0')
Buffer data legal_actions 1912: tensor([0., 0., 0., 0., 1., 0., 1., 1., 0.], device='mps:0')


/Users/eohjelle/Documents/2025-dots-and-boxes/dots-and-boxes/core/data_structures.py:91: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path, map_loca

In [3]:
from applications.tic_tac_toe.game_state import TicTacToeState
from core.implementations.Minimax import Minimax

# Creat minmax agent and expand the game tree, this will be used for evaluation later on
state = TicTacToeState()
minimax_agent = Minimax(state)
minimax_agent_root = minimax_agent.root
minimax_agent()

def minimax_agent_factory() -> Minimax:
    """
    This function returns a minimax agent that is initialized with the root of the game tree.
    """
    minimax_agent.root = minimax_agent_root
    return minimax_agent


In [4]:
# Define sweep config

sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'minimax_score',
        'goal': 'maximize'
    },
    'parameters': {
        # Optimizer parameters
        'learning_rate': {
            'distribution': 'uniform',
            'min': 0.0001,
            'max': 0.01
        },
        'weight_decay': {
            'distribution': 'uniform',
            'min': 0.001,
            'max': 0.01
        },

        # Model parameters
        'attention_layers': {
            'values': [2]
        },
        'embed_dim': {
            'values': [64]
        },
        'feedforward_dim': {
            'values': [256]
        },
        'dropout': {
            'values': [0.0]
        },
        'norm_first': {
            'values': [True]
        },
        'activation': {
            'values': ['relu']
        },

        # Training parameters
        'epochs': {
            'values': [100]
        },
        'batch_size': {
            'values': [256]
        },
        'mask_illegal_moves': {
            'values': [False]
        },
        'mask_value': {
            'values': [-10.0]
        }
    }
}

In [5]:
# Define simple training loop

import wandb
from applications.tic_tac_toe.game_state import TicTacToeState
from applications.tic_tac_toe.transformer_model import TicTacToeTransformerInterface
import torch
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader, TensorDataset, random_split
from core.benchmark import benchmark
from core.implementations.RandomAgent import RandomAgent
from core.implementations.Minimax import Minimax
from core.implementations.AlphaZero import AlphaZeroModelAgent

import os

def sweep_agent():
    with wandb.init(project="AlphaZero-TicTacToe") as run:
        config = run.config
        device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
        model_interface = TicTacToeTransformerInterface(
            device=device,
            attention_layers=config.attention_layers,
            embed_dim=config.embed_dim,
            num_heads=4,
            feedforward_dim=config.feedforward_dim,
            dropout=config.dropout,
            norm_first=config.norm_first,
            activation=config.activation
        )
        model = model_interface.model
        
        # Create optimizer
        optimizer = torch.optim.Adam(
            model.parameters(), 
            lr=config.learning_rate,
            weight_decay=config.weight_decay
        )
        
        # Create datasets
        states = buffer.states
        policy_targets = buffer.targets['policy']
        value_targets = buffer.targets['value']
        legal_actions_mask = buffer.data['legal_actions']
        
        # Create dataset and split into train/val
        dataset = TensorDataset(states, policy_targets, value_targets, legal_actions_mask)
        train_size = int(0.9 * len(dataset))
        val_size = len(dataset) - train_size
        train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
        
        # Create data loaders
        train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False)
        
        # Training loop
        best_val_loss = float('inf')
        
        for epoch in range(config.epochs):
            # Training phase
            model.train()
            train_losses = []
            policy_losses = []
            value_losses = []
            
            for batch in train_loader:
                states_batch, policy_targets_batch, value_targets_batch, legal_actions_batch = batch
                
                # Zero gradients
                optimizer.zero_grad()
                
                # Forward pass
                outputs = model(states_batch)
                policy_logits = outputs['policy']
                value_pred = outputs['value']
                
                # Apply mask for illegal moves if enabled
                if config.mask_illegal_moves:
                    policy_logits = policy_logits * legal_actions_batch + (1 - legal_actions_batch) * config.mask_value
                
                # Compute losses
                policy_loss = F.cross_entropy(policy_logits, policy_targets_batch)
                value_loss = F.mse_loss(value_pred, value_targets_batch)
                total_loss = policy_loss + value_loss
                
                # Backward pass and optimization
                total_loss.backward()
                optimizer.step()
                
                # Track metrics
                train_losses.append(total_loss.item())
                policy_losses.append(policy_loss.item())
                value_losses.append(value_loss.item())
            
            # Validation phase
            model.eval()
            val_losses = []
            val_policy_losses = []
            val_value_losses = []
            
            with torch.no_grad():
                for batch in val_loader:
                    states_batch, policy_targets_batch, value_targets_batch, legal_actions_batch = batch
                    
                    # Forward pass
                    outputs = model(states_batch)
                    policy_logits = outputs['policy']
                    value_pred = outputs['value']
                    
                    # Apply mask for illegal moves if enabled
                    if config.mask_illegal_moves:
                        policy_logits = policy_logits * legal_actions_batch + (1 - legal_actions_batch) * config.mask_value
                    
                    # Compute losses
                    policy_loss = F.cross_entropy(policy_logits, policy_targets_batch)
                    value_loss = F.mse_loss(value_pred, value_targets_batch)
                    total_loss = policy_loss + value_loss
                    
                    # Track metrics
                    val_losses.append(total_loss.item())
                    val_policy_losses.append(policy_loss.item())
                    val_value_losses.append(value_loss.item())
            
            # Calculate average metrics
            avg_train_loss = np.mean(train_losses)
            avg_val_loss = np.mean(val_losses)
            
            # Log metrics to wandb
            wandb.log({
                "epoch": epoch,
                "train_loss": avg_train_loss,
                "train_policy_loss": np.mean(policy_losses),
                "train_value_loss": np.mean(value_losses),
                "val_loss": avg_val_loss,
                "val_policy_loss": np.mean(val_policy_losses),
                "val_value_loss": np.mean(val_value_losses),
            })
            
            print(f"Epoch {epoch+1}/{config.epochs}, "
                  f"Train Loss: {avg_train_loss:.4f}, "
                  f"Val Loss: {avg_val_loss:.4f}")
            
            # # Save best model
            # if avg_val_loss < best_val_loss:
            #     best_val_loss = avg_val_loss
                
            #     # Save model checkpoint
            #     if not os.path.exists('checkpoints'):
            #         os.makedirs('checkpoints')
                    
            #     checkpoint_path = f"checkpoints/transformer_best_{run.id}.pt"
            #     torch.save({
            #         'epoch': epoch,
            #         'model_state_dict': model.state_dict(),
            #         'optimizer_state_dict': optimizer.state_dict(),
            #         'loss': best_val_loss,
            #         'config': dict(config)
            #     }, checkpoint_path)
                
            #     # Save as W&B artifact
            #     model_artifact = wandb.Artifact(
            #         f"transformer_model_{run.id}", 
            #         type="model",
            #         description=f"Best model with val_loss: {best_val_loss:.4f}"
            #     )
            #     model_artifact.add_file(checkpoint_path)
            #     run.log_artifact(model_artifact)

        # Evaluate against agents
        stats = benchmark(
            create_agent=lambda state: AlphaZeroModelAgent(
                initial_state=state,
                model=model_interface
            ),
            create_opponents={
                'random': lambda state: RandomAgent(state),
                'minimax': lambda state: minimax_agent_factory()
            },
            initial_state=lambda: TicTacToeState(),
            num_games=100
        )
        wandb.log({
            'random_win_rate': stats['random']['win_rate'],
            'minimax_win_rate': stats['minimax']['win_rate'],
            'random_draw_rate': stats['random']['draw_rate'],
            'minimax_draw_rate': stats['minimax']['draw_rate'],
            'random_loss_rate': stats['random']['loss_rate'],
            'minimax_loss_rate': stats['minimax']['loss_rate'],
            'random_score': stats['random']['win_rate'] - stats['random']['loss_rate'],
            'minimax_score': stats['minimax']['win_rate'] - stats['minimax']['loss_rate']
        })


        

In [6]:
# Start the sweep
sweep_id = wandb.sweep(sweep_config, project="AlphaZero-TicTacToe")
print(f"Sweep ID: {sweep_id}")
wandb.agent(sweep_id, function=sweep_agent, count=20) 
wandb.finish()

Create sweep with ID: mtjttrrq
Sweep URL: https://wandb.ai/eigenway/AlphaZero-TicTacToe/sweeps/mtjttrrq
Sweep ID: mtjttrrq


wandb: Agent Starting Run: 4em1d178 with config:
wandb: 	activation: relu
wandb: 	attention_layers: 2
wandb: 	batch_size: 256
wandb: 	dropout: 0
wandb: 	embed_dim: 64
wandb: 	epochs: 100
wandb: 	feedforward_dim: 256
wandb: 	learning_rate: 0.0019387894391755464
wandb: 	mask_illegal_moves: False
wandb: 	mask_value: -10
wandb: 	norm_first: True
wandb: 	weight_decay: 0.009399579829803266
wandb: Currently logged in as: eohjelle (eigenway) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch 1/100, Train Loss: 8.7050, Val Loss: 4.3128
Epoch 2/100, Train Loss: 2.9113, Val Loss: 2.8654
Epoch 3/100, Train Loss: 2.4253, Val Loss: 2.6549
Epoch 4/100, Train Loss: 2.3593, Val Loss: 2.5841
Epoch 5/100, Train Loss: 2.3192, Val Loss: 2.6640
Epoch 6/100, Train Loss: 2.3228, Val Loss: 2.6371
Epoch 7/100, Train Loss: 2.3089, Val Loss: 2.6646
Epoch 8/100, Train Loss: 2.3289, Val Loss: 2.6066
Epoch 9/100, Train Loss: 2.3242, Val Loss: 2.5992
Epoch 10/100, Train Loss: 2.3192, Val Loss: 2.6611
Epoch 11/100, Train Loss: 2.2977, Val Loss: 2.5752
Epoch 12/100, Train Loss: 2.2968, Val Loss: 2.5402
Epoch 13/100, Train Loss: 2.2599, Val Loss: 2.5264
Epoch 14/100, Train Loss: 2.2109, Val Loss: 2.4925
Epoch 15/100, Train Loss: 2.2223, Val Loss: 2.4424
Epoch 16/100, Train Loss: 2.1704, Val Loss: 2.4803
Epoch 17/100, Train Loss: 2.1345, Val Loss: 2.4614
Epoch 18/100, Train Loss: 2.1431, Val Loss: 2.4598
Epoch 19/100, Train Loss: 2.1016, Val Loss: 2.4295
Epoch 20/100, Train Loss: 2.0944, Val Lo

epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇█
minimax_draw_rate,▁
minimax_loss_rate,▁
minimax_score,▁
minimax_win_rate,▁
random_draw_rate,▁
random_loss_rate,▁
random_score,▁
random_win_rate,▁
train_loss,█▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_policy_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


...

Epoch 1/100, Train Loss: 9.2560, Val Loss: 3.7246
Epoch 2/100, Train Loss: 2.9562, Val Loss: 2.5630
Epoch 3/100, Train Loss: 2.5048, Val Loss: 2.5525
Epoch 4/100, Train Loss: 2.4067, Val Loss: 2.4658
Epoch 5/100, Train Loss: 2.3297, Val Loss: 2.3991
Epoch 6/100, Train Loss: 2.2680, Val Loss: 2.4064
Epoch 7/100, Train Loss: 2.2802, Val Loss: 2.3091
Epoch 8/100, Train Loss: 2.2331, Val Loss: 2.3145
Epoch 9/100, Train Loss: 2.2019, Val Loss: 2.2319
Epoch 10/100, Train Loss: 2.1736, Val Loss: 2.2747
Epoch 11/100, Train Loss: 2.1491, Val Loss: 2.2042
Epoch 12/100, Train Loss: 2.1006, Val Loss: 2.1545
Epoch 13/100, Train Loss: 2.0693, Val Loss: 2.1373
Epoch 14/100, Train Loss: 2.0818, Val Loss: 2.1185
Epoch 15/100, Train Loss: 2.0592, Val Loss: 2.1113
Epoch 16/100, Train Loss: 2.0501, Val Loss: 2.1163
Epoch 17/100, Train Loss: 2.0578, Val Loss: 2.0843
Epoch 18/100, Train Loss: 2.0341, Val Loss: 2.1088
Epoch 19/100, Train Loss: 2.0632, Val Loss: 2.0964
Epoch 20/100, Train Loss: 2.0302, Val Lo

epoch,▁▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇███
minimax_draw_rate,▁
minimax_loss_rate,▁
minimax_score,▁
minimax_win_rate,▁
random_draw_rate,▁
random_loss_rate,▁
random_score,▁
random_win_rate,▁
train_loss,█▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_policy_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


wandb: Agent Starting Run: iatpzvs4 with config:
wandb: 	activation: relu
wandb: 	attention_layers: 2
wandb: 	batch_size: 256
wandb: 	dropout: 0
wandb: 	embed_dim: 64
wandb: 	epochs: 100
wandb: 	feedforward_dim: 256
wandb: 	learning_rate: 0.009314506414361929
wandb: 	mask_illegal_moves: False
wandb: 	mask_value: -10
wandb: 	norm_first: True
wandb: 	weight_decay: 0.008564587216366352


Epoch 1/100, Train Loss: 11.0705, Val Loss: 3.8094
Epoch 2/100, Train Loss: 3.2318, Val Loss: 2.9071
Epoch 3/100, Train Loss: 2.7688, Val Loss: 2.6740
Epoch 4/100, Train Loss: 2.6211, Val Loss: 2.5917
Epoch 5/100, Train Loss: 2.5301, Val Loss: 2.5288
Epoch 6/100, Train Loss: 2.4512, Val Loss: 2.4905
Epoch 7/100, Train Loss: 2.4145, Val Loss: 2.4508
Epoch 8/100, Train Loss: 2.3718, Val Loss: 2.4364
Epoch 9/100, Train Loss: 2.3665, Val Loss: 2.4406
Epoch 10/100, Train Loss: 2.3735, Val Loss: 2.4477
Epoch 11/100, Train Loss: 2.4767, Val Loss: 2.3892
Epoch 12/100, Train Loss: 2.3481, Val Loss: 2.4120
Epoch 13/100, Train Loss: 2.2975, Val Loss: 2.3960
Epoch 14/100, Train Loss: 2.2553, Val Loss: 2.3768
Epoch 15/100, Train Loss: 2.2289, Val Loss: 2.3078
Epoch 16/100, Train Loss: 2.1832, Val Loss: 2.2432
Epoch 17/100, Train Loss: 2.1455, Val Loss: 2.2407
Epoch 18/100, Train Loss: 2.1329, Val Loss: 2.1885
Epoch 19/100, Train Loss: 2.1081, Val Loss: 2.2843
Epoch 20/100, Train Loss: 2.1207, Val L

epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▄▄▄▄▄▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇█████
minimax_draw_rate,▁
minimax_loss_rate,▁
minimax_score,▁
minimax_win_rate,▁
random_draw_rate,▁
random_loss_rate,▁
random_score,▁
random_win_rate,▁
train_loss,▇▇▆▆▆▆▆▆▅▅▅▅▅▅▅██████▇▅▅▅▅▅▅▅▅▅▃▂▂▂▁▁▁▁▁
train_policy_loss,█▅▅▄▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁
